In [59]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from IPython.display import display

%matplotlib inline

In [60]:
# Read data from .csv files 

loc = "voetbal_new/"

raw_cards_file = pd.read_csv (loc + '_cards.csv')
raw_goals_file = pd.read_csv (loc + '_goals.csv')
raw_matches_file = pd.read_csv (loc + '_matches.csv')
raw_players_file = pd.read_csv (loc + '_players.csv')
raw_subs_file = pd.read_csv (loc + '_subs.csv')

In [61]:
# function to remove duplicates 
def Duplicates(rawName , colName: str, previousName: str, newName: str ):
        if colName == "club":
                rawName["home"].replace({previousName: newName }, inplace=True)
                rawName["away"].replace({previousName: newName }, inplace=True)
                rawName["team"].replace({previousName: newName }, inplace=True)
        elif colName == "match":
                rawName["home"].replace({previousName: newName }, inplace=True)
                rawName["away"].replace({previousName: newName }, inplace=True)
        elif colName == "card": 
                rawName["type"].replace({previousName: newName }, inplace=True)
        else:
                print("nothing")

In [95]:
def searchTeam(teams, teamName):
    for (k,i) in teams.items():
        if k == teamName:
            return (k, i)

## cards

In [62]:
df_card_filtered = raw_cards_file[(raw_cards_file['type'] == 'rode kaart') | 
                                  (raw_cards_file['type'] == 'gele kaart') |
                                  (raw_cards_file['type'] == 'gele') |
                                  (raw_cards_file['type'] == 'kaartgele') |
                                  (raw_cards_file['type'] == 'rode') |
                                  (raw_cards_file['type'] == 'kaartrode')  ]
            
df_card_filtered["type"].replace({"rode kaart": 2, "gele kaart": 1, "gele": 1 , "kaartgele" : 1 , "rode":2, "kaartrode" : 2 }, inplace=True)

filterHomeTeam = df_card_filtered[ (df_card_filtered['home'] == df_card_filtered['team'])]
filterAwayTeam = df_card_filtered[ (df_card_filtered['away'] == df_card_filtered['team'])]

C:\Users\malja\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [63]:
# Away yellow cards
filterAwayYellowCard = filterAwayTeam[(filterAwayTeam['type'] == 1)]
groupByAwayYellowCard = filterAwayYellowCard.groupby('away').sum()
groupByAwayYellowCard['type'].head(10)

print(f'{len(filterAwayYellowCard.index)} yellow cards is made by away team ')

# filterAwayYellowCard = filterAwayTeam[(filterAwayTeam['type'] == 1)]
# groupByAwayYellowCard = filterAwayYellowCard.groupby('away').sum()
# groupByAwayYellowCard['type'].head(10)

17989 yellow cards is made by away team 


In [64]:
# Away red cards
filterAwayRedCard = filterAwayTeam[(filterAwayTeam['type'] == 2)]
print(f'{len(filterAwayRedCard.index)} red cards is made by away team ')

# filterAwayRedCard = filterAwayTeam[(filterAwayTeam['type'] == 2)]
# groupByAwayRedCard = filterAwayRedCard.groupby('away').sum()
# groupByAwayRedCard['type'].head(10)

680 red cards is made by away team 


In [65]:
# Home yellow cards
filterHomeYellowCard = filterHomeTeam[(filterHomeTeam['type'] == 1)]
groupByHomeYellowCard = filterHomeYellowCard.groupby('home').sum()
groupByHomeYellowCard

print(f'{len(filterHomeYellowCard.index)} yellow cards is made by home team ')

# filterHomeYellowCard = filterHomeTeam[(filterHomeTeam['type'] == 1)]
# groupByHomeYellowCard = filterHomeYellowCard.groupby('home').sum()
# groupByHomeYellowCard

12632 yellow cards is made by home team 


In [66]:
# Home red cards 
filterHomeRedCard = filterHomeTeam[(filterHomeTeam['type'] == 2)]
print(f'{len(filterHomeRedCard.index)} red cards is made by home team ')

# filterHomeRedCard = filterHomeTeam[(filterHomeTeam['type'] == 2)]
# groupByHomeRedCard = filterHomeRedCard.groupby('home').sum()
# groupByHomeRedCard

416 red cards is made by home team 


## Goals

In [67]:
# Method used to overwrite values found in column 'assist'
def assist(value):
    if pd.notna(value):
        return 1
    else:
        return 0

In [68]:
raw_goals_file["home"].replace({"afc-ajax": "ajax-amsterdam" }, inplace=True)
raw_goals_file["away"].replace({"afc-ajax": "ajax-amsterdam" }, inplace=True)
raw_goals_file["team"].replace({"afc-ajax": "ajax-amsterdam" }, inplace=True)
filtered_assists = raw_goals_file.drop(columns=['new_score','type','minute','round'])
filtered_assists['assist'] = raw_goals_file['assist'].apply(assist)
filtered_assists['player'] = raw_goals_file['player'].apply(assist)
filtered_assists

,Unnamed: 0,saison,home,away,team,player,assist
0,0,1956-1957,ajax-amsterdam,bvc-amsterdam,ajax-amsterdam,1,1
1,1,1956-1957,ajax-amsterdam,bvc-amsterdam,bvc-amsterdam,1,0
2,2,1956-1957,ajax-amsterdam,bvc-amsterdam,bvc-amsterdam,1,0
3,3,1956-1957,ajax-amsterdam,bvc-amsterdam,ajax-amsterdam,1,0
4,4,1956-1957,ajax-amsterdam,bvc-amsterdam,bvc-amsterdam,1,0
...,...,...,...,...,...,...,...
60982,757,2021-2022,willem-ii,sc-cambuur,sc-cambuur,1,0
60983,758,2021-2022,willem-ii,sparta-rotterdam,sparta-rotterdam,1,1
60984,759,2021-2022,willem-ii,sparta-rotterdam,sparta-rotterdam,1,1
60985,760,2021-2022,willem-ii,sparta-rotterdam,sparta-rotterdam,1,1


In [69]:
# Home assist and goals
filterAssist = filtered_assists[(filtered_assists['assist'] == 1)]
groupByHomeAssist = filterAssist.groupby('home')['assist'].sum()
print(groupByHomeAssist.head())

# goals 
groupByHomeGoals = filtered_assists.groupby('home')['player'].sum()
print(groupByHomeGoals.head())

home
ado-den-haag       617
afc-dws             17
ajax-amsterdam    1600
az-alkmaar         830
blauw-wit            2
Name: assist, dtype: int64
home
ado-den-haag      2405
afc-dws            604
ajax-amsterdam    4307
az-alkmaar        2234
blauw-wit          348
Name: player, dtype: int64


In [70]:
# Away assist and goals 
groupByAwayAssist = filterAssist.groupby('away')['assist'].sum()
print(groupByAwayAssist.head())

# goals 
groupByAwayGoals = filtered_assists.groupby('away')['player'].sum()
print(groupByAwayGoals.head(101))

away
ado-den-haag       612
afc-dws             17
ajax-amsterdam    1316
az-alkmaar         843
blauw-wit            8
Name: assist, dtype: int64
away
ado-den-haag           2437
afc-dws                 628
ajax-amsterdam         3561
az-alkmaar             2399
blauw-wit               387
bvc-amsterdam           121
cambuur-leeuwarden      140
de-graafschap          1093
de-volewijckers         150
excelsior-rotterdam     139
fc-amsterdam            287
fc-den-bosch            824
fc-dordrecht            357
fc-eindhoven            169
fc-emmen                139
fc-groningen           2733
fc-twente              2941
fc-utrecht             3385
fc-volendam            1310
fc-wageningen           120
feyenoord              3224
feyenoord-rotterdam     157
fortuna-54              502
fortuna-sittard        1277
go-ahead-eagles        1573
go-ahead-eagles_2         3
helmond-sport           139
heracles-almelo        1141
hfc-haarlem             869
holland-sport           230
mvv     

## Matches 

In [71]:
Duplicates(raw_matches_file, "match", "afc-ajax", "ajax-amsterdam")
Duplicates(raw_matches_file, "match", "willem-ii", "willem-ii-tilburg")
Duplicates(raw_matches_file, "match", "mvv", "mvv-maastricht")
Duplicates(raw_matches_file, "match", "vitesse", "vitesse-arnhem")
Duplicates(raw_matches_file, "match", "sc-cambuur", "cambuur-leeuwarden")
Duplicates(raw_matches_file, "match", "feyenoord", "feyenoord-rotterdam")
Duplicates(raw_matches_file, "match", "go-ahead-eagles_2", "go-ahead-eagles")

filtered_matches = raw_matches_file.drop(columns=['stadium','home_first_card','home_last_card',
                                                 'away_first_card', 'time','date',
                                                 'spectators', 'ref','linesman1','linesman2', 'away_last_card',
                                                 'home_yellow', 'home_yelred', 'away_yellow','first_goal', 'last_goal',
                                                  'away_red','home_red', 'away_yelred'])

filtered_matches.dropna()
filtered_matches

,Unnamed: 0,saison,round,home,away,home_score,away_score
0,0,1956-1957,16,ajax-amsterdam,bvc-amsterdam,2,3
1,1,1956-1957,27,ajax-amsterdam,fc-den-bosch,1,0
2,2,1956-1957,15,ajax-amsterdam,fc-utrecht,1,2
3,3,1956-1957,25,ajax-amsterdam,feyenoord-rotterdam,1,0
4,4,1956-1957,13,ajax-amsterdam,fortuna-54,0,2
...,...,...,...,...,...,...,...
19925,301,2021-2022,21,willem-ii-tilburg,rkc-waalwijk,3,1
19926,302,2021-2022,16,willem-ii-tilburg,cambuur-leeuwarden,1,3
19927,303,2021-2022,25,willem-ii-tilburg,sc-heerenveen,0,0
19928,304,2021-2022,12,willem-ii-tilburg,sparta-rotterdam,0,3


In [72]:
WinningHomeTeamFilter = filtered_matches[ (filtered_matches['home_score'] > filtered_matches['away_score'])]
# WinningHomeTeamFilter
print(f'The Home team won {len(WinningHomeTeamFilter.index)} matches')

WinningAwayTeamFilter = filtered_matches[ (filtered_matches['home_score'] < filtered_matches['away_score'])]
# WinningAwayTeamFilter
print(f'The Away team won {len(WinningAwayTeamFilter.index)} matches')

DrawTeamFilter = filtered_matches[ (filtered_matches['home_score'] == filtered_matches['away_score'])]
# WinningAwayTeamFilter
print(f'The draw was in {len(DrawTeamFilter.index)} matches')

The Home team won 9480 matches
The Away team won 5431 matches
The draw was in 5019 matches


## Players

In [73]:
# TODO research            
raw_players_file["started"].replace({True: 1, False: 0}, inplace=True)

filterHomeTeamPlayers = raw_players_file[ (raw_players_file['home'] == raw_players_file['team'])]
filterAwayTeamPlayers = raw_players_file[ (raw_players_file['away'] == raw_players_file['team'])]


filterStartedPlayersHome = filterHomeTeamPlayers[ (filterHomeTeamPlayers['started'] == 1)]
print(f'{len(filterStartedPlayersHome.index)} players play home')
filterNotStartedPlayersHome = filterHomeTeamPlayers[ (filterHomeTeamPlayers['started'] == 0)]
print(f'{len(filterNotStartedPlayersHome.index)} players did not play home')

filterStartedPlayersAway = filterAwayTeamPlayers[ (filterAwayTeamPlayers['started'] == 1)]
print(f'{len(filterStartedPlayersAway.index)} players play away')
filterNotStartedPlayersAway = filterAwayTeamPlayers[ (filterAwayTeamPlayers['started'] == 0)]
print(f'{len(filterNotStartedPlayersAway.index)} players did not play away')

213182 players play home
50207 players did not play home
217723 players play away
51952 players did not play away


## SUBS

In [74]:
# TODO research 
raw_subs_file

,Unnamed: 0,saison,round,home,away,team,player,minute,in
0,0,1956-1957,16,afc-ajax,bvc-amsterdam,afc-ajax,rinus-michels,70',False
1,1,1956-1957,16,afc-ajax,bvc-amsterdam,afc-ajax,loek-den-edel,70',True
2,2,1956-1957,27,afc-ajax,fc-den-bosch,afc-ajax,rinus-michels,46',False
3,3,1956-1957,27,afc-ajax,fc-den-bosch,afc-ajax,piet-ouderland,46',True
4,4,1956-1957,15,afc-ajax,fc-utrecht,afc-ajax,wim-anderiesen,83',False
...,...,...,...,...,...,...,...,...,...
129190,4257,2021-2022,30,willem-ii,vitesse,willem-ii,elton-kabangu,73',True
129191,4258,2021-2022,30,willem-ii,vitesse,vitesse,million-manhoef,84',False
129192,4259,2021-2022,30,willem-ii,vitesse,vitesse,daan-huisman,72',False
129193,4260,2021-2022,30,willem-ii,vitesse,vitesse,miliano-jonathans,84',True


In [75]:
# function used to get the last season
def toOneSeason(season):
    return season[5:len(season)]

In [76]:
Duplicates(raw_cards_file, "club", "afc-ajax", "ajax-amsterdam")
Duplicates(raw_cards_file, "club", "willem-ii", "willem-ii-tilburg")
Duplicates(raw_cards_file, "club", "mvv", "mvv-maastricht")
Duplicates(raw_cards_file, "club", "vitesse", "vitesse-arnhem")
Duplicates(raw_cards_file, "club", "sc-cambuur", "cambuur-leeuwarden")
Duplicates(raw_cards_file, "club", "feyenoord", "feyenoord-rotterdam")
Duplicates(raw_cards_file, "club", "go-ahead-eagles_2", "go-ahead-eagles")

Duplicates(raw_cards_file, "card", "rode kaart", "red-card")
Duplicates(raw_cards_file, "card", "rode", "red-card")
Duplicates(raw_cards_file, "card", "kaartrode", "red-card")
Duplicates(raw_cards_file, "card", "gele kaart", "yellow-card")
Duplicates(raw_cards_file, "card", "gele", "yellow-card")
Duplicates(raw_cards_file, "card", "kaartgele", "yellow-card")
Duplicates(raw_cards_file, "card", "geelrode kaart", "red-card")
raw_cards_file.drop(columns=['player','type'])
raw_cards_file['saison'] = raw_cards_file['saison'].apply(toOneSeason)
raw_cards_file['saison'] = raw_cards_file['saison'].astype(int)


df_cards = pd.DataFrame()
season = raw_cards_file['saison']
round = raw_cards_file['round']
home = raw_cards_file['home']
away = raw_cards_file['away']
team = raw_cards_file['team']
type = raw_cards_file['type']
yellowCards = raw_cards_file['type']
redCards = raw_cards_file['type']
home_cards = raw_cards_file['type']
away_cards = raw_cards_file['type']



df_cards['season'] =  season
df_cards['round'] =  round
df_cards['home'] =  home
df_cards['away'] =  away
df_cards['team'] =  team
df_cards['type'] =  type
df_cards['yellowCards'] =  yellowCards
df_cards['redCards'] =  redCards
df_cards['home_cards'] =  home_cards
df_cards['away_cards'] =  away_cards
df_cards["yellowCards"].replace({"red-card": 0, "yellow-card": 1 }, inplace=True)
df_cards["redCards"].replace({"red-card": 1, "yellow-card": 0 }, inplace=True)

df_cards.loc[(df_cards["home_cards"] == "red-card") & (df_cards["home"] == df_cards["team"]), "home_cards"] = "red"
df_cards.loc[(df_cards["away_cards"] == "red-card") & (df_cards["away"] == df_cards["team"]), "away_cards"] = "red"
df_cards.loc[(df_cards["home_cards"] == "yellow-card") & (df_cards["home"] == df_cards["team"]), "home_cards"] = "yellow"
df_cards.loc[(df_cards["away_cards"] == "yellow-card") & (df_cards["away"] == df_cards["team"]), "away_cards"] = "yellow"
df_cards["home_cards"].replace({"red-card": 0, "yellow-card": 0, "yellow": 1,  "red": 2 }, inplace=True)
df_cards["away_cards"].replace({"red-card": 0, "yellow-card": 0, "yellow": 1,  "red": 2 }, inplace=True)


df_cards[(df_cards['season'] >= 2012)]
df_cards

,season,round,home,away,team,type,yellowCards,redCards,home_cards,away_cards
0,1963,8,heracles-almelo,fc-twente,fc-twente,red-card,0,1,0,2
1,1972,5,fc-groningen,ajax-amsterdam,fc-groningen,red-card,0,1,2,0
2,1972,5,fc-groningen,ajax-amsterdam,ajax-amsterdam,red-card,0,1,0,2
3,1973,34,ajax-amsterdam,az-alkmaar,ajax-amsterdam,yellow-card,1,0,1,0
4,1973,34,ajax-amsterdam,az-alkmaar,az-alkmaar,yellow-card,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
32412,2022,12,willem-ii-tilburg,sparta-rotterdam,sparta-rotterdam,yellow-card,1,0,0,1
32413,2022,30,willem-ii-tilburg,vitesse-arnhem,vitesse-arnhem,yellow-card,1,0,0,1
32414,2022,30,willem-ii-tilburg,vitesse-arnhem,vitesse-arnhem,yellow-card,1,0,0,1
32415,2022,30,willem-ii-tilburg,vitesse-arnhem,vitesse-arnhem,yellow-card,1,0,0,1


In [77]:
df_cards_new = pd.DataFrame()
season = df_cards['season']
round = df_cards['round']
home = df_cards['home']
away = df_cards['away']
team = df_cards['team']
type = df_cards['type']
yellowCards = df_cards['yellowCards']
redCards = df_cards['redCards']
home_cards = df_cards['home_cards']
away_cards = df_cards['away_cards']


df_cards_new['season'] =  season
df_cards_new['round'] =  round
df_cards_new['home'] =  home
df_cards_new['away'] =  away
df_cards_new['team'] =  team
df_cards_new['type'] =  type
df_cards_new['yellowCards'] =  yellowCards
df_cards_new['redCards'] =  redCards
df_cards_new['yellow_card_home'] =  home_cards
df_cards_new['yellow_card_away'] =  away_cards
df_cards_new['red_card_home'] =  home_cards
df_cards_new['red_card_away'] =  away_cards

df_cards_new["yellow_card_home"].replace({2: 0, 1: 1, 0: 0}, inplace=True)
df_cards_new["yellow_card_away"].replace({2: 0, 1: 1, 0: 0}, inplace=True)
df_cards_new["red_card_home"].replace({2: 1, 1: 0, 0: 0}, inplace=True)
df_cards_new["red_card_away"].replace({2: 1, 1: 0, 0: 0}, inplace=True)

df_cards_new

,season,round,home,away,team,type,yellowCards,redCards,yellow_card_home,yellow_card_away,red_card_home,red_card_away
0,1963,8,heracles-almelo,fc-twente,fc-twente,red-card,0,1,0,0,0,1
1,1972,5,fc-groningen,ajax-amsterdam,fc-groningen,red-card,0,1,0,0,1,0
2,1972,5,fc-groningen,ajax-amsterdam,ajax-amsterdam,red-card,0,1,0,0,0,1
3,1973,34,ajax-amsterdam,az-alkmaar,ajax-amsterdam,yellow-card,1,0,1,0,0,0
4,1973,34,ajax-amsterdam,az-alkmaar,az-alkmaar,yellow-card,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32412,2022,12,willem-ii-tilburg,sparta-rotterdam,sparta-rotterdam,yellow-card,1,0,0,1,0,0
32413,2022,30,willem-ii-tilburg,vitesse-arnhem,vitesse-arnhem,yellow-card,1,0,0,1,0,0
32414,2022,30,willem-ii-tilburg,vitesse-arnhem,vitesse-arnhem,yellow-card,1,0,0,1,0,0
32415,2022,30,willem-ii-tilburg,vitesse-arnhem,vitesse-arnhem,yellow-card,1,0,0,1,0,0


In [78]:
groupByAwayYellowCard = df_cards_new.groupby(['season', 'round','home', 'away'], as_index=False).agg(yellowCards = ('yellowCards', 'sum'), 
                                                                                redCards = ('redCards', 'sum'),
                                                                                yellowCardHome = ('yellow_card_home', 'sum'),
                                                                                yellowCardAway = ('yellow_card_away', 'sum'),
                                                                                redCardHome = ('red_card_home', 'sum'),
                                                                                redCardAway = ('red_card_away', 'sum'))
groupByAwayYellowCard

,season,round,home,away,yellowCards,redCards,yellowCardHome,yellowCardAway,redCardHome,redCardAway
0,1963,8,heracles-almelo,fc-twente,0,1,0,0,0,1
1,1972,5,fc-groningen,ajax-amsterdam,0,2,0,0,1,1
2,1973,1,ado-den-haag,sparta-rotterdam,2,0,0,2,0,0
3,1973,1,feyenoord-rotterdam,nec-nijmegen,2,0,1,1,0,0
4,1973,1,nac-breda,go-ahead-eagles,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
11166,2022,30,fc-twente,sparta-rotterdam,3,0,0,3,0,0
11167,2022,30,feyenoord-rotterdam,fc-utrecht,3,0,1,2,0,0
11168,2022,30,nec-nijmegen,ajax-amsterdam,2,0,0,2,0,0
11169,2022,30,rkc-waalwijk,pec-zwolle,3,0,2,1,0,0


In [79]:
#Function used for the matches:
def ClassifyMatches(row):
    if(row.home_score > row.away_score):
        return 1
    elif(row.home_score < row.away_score):
        return -1
    else:
        return 0

In [80]:
filtered_matches['saison'] = filtered_matches['saison'].apply(toOneSeason)
filtered_matches['saison'] = filtered_matches['saison'].astype(int)

df_MatchesForAjaxHost = pd.DataFrame()
season = filtered_matches['saison']
round = filtered_matches['round']
homeTeamAjax = filtered_matches['home']
awayTeamAjax = filtered_matches['away']
scoresAjaxHome = filtered_matches['home_score']
awayTeamScoresAjax = filtered_matches['away_score']

df_MatchesForAjaxHost['season'] = season
df_MatchesForAjaxHost['round'] = round
df_MatchesForAjaxHost['home'] = homeTeamAjax
df_MatchesForAjaxHost['away'] = awayTeamAjax
df_MatchesForAjaxHost['home_score'] = scoresAjaxHome
df_MatchesForAjaxHost['away_score'] = awayTeamScoresAjax
df_MatchesForAjaxHost['winning_team'] = df_MatchesForAjaxHost.apply(lambda row: ClassifyMatches(row),axis=1)
df_MatchesForAjaxHost[(df_MatchesForAjaxHost['season'] >= 2012) & 
                                        (df_MatchesForAjaxHost['home_score'] != '-') &
                                        (df_MatchesForAjaxHost['away_score'] != '-')]
df_MatchesForAjaxHost

,season,round,home,away,home_score,away_score,winning_team
0,1957,16,ajax-amsterdam,bvc-amsterdam,2,3,-1
1,1957,27,ajax-amsterdam,fc-den-bosch,1,0,1
2,1957,15,ajax-amsterdam,fc-utrecht,1,2,-1
3,1957,25,ajax-amsterdam,feyenoord-rotterdam,1,0,1
4,1957,13,ajax-amsterdam,fortuna-54,0,2,-1
...,...,...,...,...,...,...,...
19925,2022,21,willem-ii-tilburg,rkc-waalwijk,3,1,1
19926,2022,16,willem-ii-tilburg,cambuur-leeuwarden,1,3,-1
19927,2022,25,willem-ii-tilburg,sc-heerenveen,0,0,0
19928,2022,12,willem-ii-tilburg,sparta-rotterdam,0,3,-1


In [81]:
# df_merge = pd.merge(df_MatchesForAjaxHost, groupByAwayYellowCard, on='season',how='outer')
df_merge= pd.merge(df_MatchesForAjaxHost,groupByAwayYellowCard,how='inner',left_on=['season','round','home', 'away'],right_on=['season','round','home', 'away'])
df_test = df_merge[(df_merge['home_score'] != '-') |
                    (df_merge['away_score'] != '-')]
df_test["home_score"].replace({'-': '0'}, inplace=True)
df_test["away_score"].replace({'- afgebr.': '0'}, inplace=True)
df_test["away_score"].replace({'2 Toeg.': '0'}, inplace=True)
                                       
df_test.tail(10)

,season,round,home,away,home_score,away_score,winning_team,yellowCards,redCards,yellowCardHome,yellowCardAway,redCardHome,redCardAway
11161,2022,1,willem-ii-tilburg,feyenoord-rotterdam,0,4,-1,1,1,1,0,1,0
11162,2022,10,willem-ii-tilburg,fortuna-sittard,1,1,0,3,0,1,2,0,0
11163,2022,14,willem-ii-tilburg,go-ahead-eagles,0,1,-1,2,0,1,1,0,0
11164,2022,18,willem-ii-tilburg,nec-nijmegen,0,1,-1,0,1,0,0,1,0
11165,2022,7,willem-ii-tilburg,psv-eindhoven,2,1,1,3,0,1,2,0,0
11166,2022,21,willem-ii-tilburg,rkc-waalwijk,3,1,1,2,0,0,2,0,0
11167,2022,16,willem-ii-tilburg,cambuur-leeuwarden,1,3,-1,2,0,1,1,0,0
11168,2022,25,willem-ii-tilburg,sc-heerenveen,0,0,0,2,0,2,0,0,0
11169,2022,12,willem-ii-tilburg,sparta-rotterdam,0,3,-1,4,0,1,3,0,0
11170,2022,30,willem-ii-tilburg,vitesse-arnhem,1,0,1,4,0,0,4,0,0


In [82]:
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.model_selection import train_test_split

label = LabelEncoder()

In [83]:
ids = label.fit_transform(df_test['away_score'])
mappingHome = dict(zip(label.classes_, range(len(label.classes_))))

In [96]:
ids = label.fit_transform(df_merge['home'])
mappingHome = dict(zip(label.classes_, range(len(label.classes_))))
print(list(mappingHome.keys())[list(mappingHome.values()).index(1)])

df_test["home"] = label.fit_transform(df_test['home'])
df_test["away"] = label.fit_transform(df_test['away'])

ajax-amsterdam


In [97]:
searchTeam(mappingHome, "ajax-amsterdam")

('ajax-amsterdam', 1)

In [85]:
X = df_test[["home","away"]]
y = df_test["winning_team"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [86]:
winning_team = tree.DecisionTreeClassifier()
winning_team.fit(X_train,y_train)

winning_team.score(X_test,y_test)

0.4680089485458613

In [93]:
winning_team.predict([[1, 26]])

array([-1], dtype=int64)